In [118]:
import wandb
import Framework.Confounder_Injection as CI
import Framework.Models as Models
import importlib
importlib.reload(Models)
importlib.reload(CI)
import pandas as pd
import pathlib
import time


In "Validation of synthetic data" the case were the training_data was completely confounded was tested to show that neural networks indeed fit to confounding factors in images. Now the hypothesis is that if we have a small set of unconfounded data we can either use a confounder-free neural network or a DANN to unlearn the confounders. For establishing a performance baseline we need to test the SimpleConv on a dataset consisting of the confounded set and the small unconfounded set, otherwise the conditions would not be equal for the different networks.

# No confounders in target and test_data

In this case there are 512 samples from the source domain (with correlating confounders) and a varying number of samples (16 or 64) from the target domain (with no confounders).

In this case the network is able to achieve the highest possible accuracy. When the confounder is absent the network seems to be able to use the real features for distinguishing the classes and when the confounder is present the network can use it to achieve higher accuracy.

In [ ]:
CI.sync_wandb_data("BrNet on br_net data")

In [ ]:
# define filename
dir = pathlib.Path().resolve()
file = dir / "BrNet on br_net data.json"

# create plot-object for later use
p = CI.plot_from_csv()
# get newest date
#date = "2022.7.19 14:11:9"
date = CI.get_dates(file)[0]
date = "2022.8.2 10:15:26"
print(date)

### With 16 training-samples from target population

The accuracy with more samples is the same but the network converges faster.

In [ ]:
config_filter = {"confounder_strength":0, "target_domain_samples":16, "de_correlate_confounder_target": False, "date": date}
p.accuracy_vs_epoch(file, config_filter, groupby="model")

In [ ]:
config_filter = {"confounder_strength":1, "target_domain_samples":16, "de_correlate_confounder_target": False, "date": date}
p.accuracy_vs_epoch(file, config_filter, groupby="model")

# Correlated

In [ ]:
config_filter = {"confounder_strength":1, "target_domain_samples":0, "de_correlate_confounder_target": False, "target_domain_confounding":1, "date": date}
p.accuracy_vs_epoch(file, config_filter, groupby="model")

# De-correlated confounders in target- and test-data

In this case there are confounders present in the data from target domain and test-set but they are de-correlated with the real features (they are rather distributed by pure chance). This is more representative of real world examples.

In this case the SimpleConv's accuracy diminishes when the confounding strength increases, contrary to the case before. Again the network uses the confounder as approximation, if present. But this time the confounder gives wrong clues about the classes and therefore the network's accuracy drops heavily.

### With 16 training-samples from target population

In [ ]:
config_filter = {"confounder_strength":1, "target_domain_samples":16, "de_correlate_confounder_target": True, "date": date}
p.accuracy_vs_epoch(file, config_filter, groupby="model")

### With 128 training-samples from target population

In [ ]:
config_filter = {"confounder_strength":1, "target_domain_samples":128, "de_correlate_confounder_target": True, "date": date}
p.accuracy_vs_epoch(file, config_filter, groupby="model")

In [ ]:
#c.show_time()